## Pre-processing & Training Data Development :

In this step, first diffrent NLP model will train to see which one has the best performance for the sentiment prediction of the reviews.

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV,cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer


In [3]:
review_db = pd.read_csv('/Users/Amin/Documents/GitHub/Review-Sentiment-Analysis-with-Recommendation-System/data/clean_db.csv')

In [5]:
review_db.head()

,reviewer,movie,review_summary,review_date,review_detail,lable,director,genre_1,genre_2,genre_3
0,OriginalMovieBuff21,Kill Bill: Vol. 2,good follow up that answers all the questions,2005-07-24,after seeing tarantino's kill bill vol: 1 i go...,LOVE IT,Quentin Tarantino,Action,Crime,Thriller
1,Bogmeister,Kill Bill: Vol. 2,the bride ends her rampage; we applaud,2005-08-15,the 2nd half of tarantino's tale of bloody rev...,LOVE IT,Quentin Tarantino,Action,Crime,Thriller
2,departed07,Kill Bill: Vol. 2,quentin tarantino's best since pulp fiction,2005-08-26,the bride is back and ready to kick ass in thi...,LOVE IT,Quentin Tarantino,Action,Crime,Thriller
3,Angeneer,Kill Bill: Vol. 2,quentin made me eat my hat,2005-09-08,i'm very happy to admit that tarantino proved ...,LOVE IT,Quentin Tarantino,Action,Crime,Thriller
4,LoneWolfAndCub,Kill Bill: Vol. 2,great ending to qt's tale of revenge,2005-09-07,kill bill volume 2 (directed by quentin tarant...,LOVE IT,Quentin Tarantino,Action,Crime,Thriller


In [6]:
X_train,y_train,X_test,y_test = train_test_split(review_db.review_detail,review_db.lable,test_size=0.2, 
                                                    random_state=47)

In [15]:
X_train.head()

111429    most franchises are showing signs of flagging ...
176404    let me start by saying that this was a very go...
286447    the story of jesse james is the richest source...
172009    it's not everyday one finds a teen-slasher fli...
321866    "cassandra's dream" reminds me of those one-of...
Name: review_detail, dtype: object

In [14]:
token = word_tokenize(X_train.iloc[0])
token

['most',
 'franchises',
 'are',
 'showing',
 'signs',
 'of',
 'flagging',
 'by',
 'the',
 '3rd',
 'film',
 'but',
 'the',
 'rocky',
 'series',
 'defies',
 'this',
 'as',
 'rocky',
 '3',
 'is',
 'another',
 'absorbing',
 'film',
 '.',
 'rocky',
 'has',
 'made',
 'it',
 'to',
 'world',
 'champion',
 'and',
 'is',
 'enjoying',
 'the',
 'fruits',
 'of',
 'success',
 '.',
 'that',
 'is',
 'until',
 'clubber',
 'lang',
 'comes',
 'onto',
 'the',
 'scene',
 '(',
 'an',
 'impressively',
 'aggressive',
 'mr',
 't',
 ')',
 '.',
 'this',
 'entry',
 'features',
 'more',
 'fights',
 'and',
 'a',
 'charity',
 'boxing',
 'v',
 'wwe',
 'match',
 'between',
 'rocky',
 'and',
 'hulk',
 'hogan',
 '.',
 'it',
 "'s",
 'an',
 'explosive',
 'mix',
 'and',
 'this',
 'film',
 'does',
 "n't",
 'slow',
 'down',
 'for',
 'a',
 'minute',
 '.',
 'not',
 'that',
 'it',
 "'s",
 'all',
 'action',
 'as',
 'it',
 'does',
 'feature',
 'one',
 'of',
 'the',
 'saddest',
 'scenes',
 'in',
 'all',
 'the',
 'rocky',
 'films',

In [18]:
def punc_clean(text):
    import string as st
    a=[w for w in text if w not in st.punctuation]
    return ''.join(a)
X_train = X_train.apply(punc_clean)
X_test = X_test.apply(punc_clean)

In [19]:
def remove_stopword(text):
    stopword=nltk.corpus.stopwords.words('english')
    stopword.remove('not')
    a=[w for w in nltk.word_tokenize(text) if w not in stopword]
    return ' '.join(a)
X_train = X_train.apply(remove_stopword)
X_test = X_test.apply(remove_stopword)

In [21]:
token = word_tokenize(X_train.iloc[0])
token

['franchises',
 'showing',
 'signs',
 'flagging',
 '3rd',
 'film',
 'rocky',
 'series',
 'defies',
 'rocky',
 '3',
 'another',
 'absorbing',
 'film',
 'rocky',
 'made',
 'world',
 'champion',
 'enjoying',
 'fruits',
 'success',
 'clubber',
 'lang',
 'comes',
 'onto',
 'scene',
 'impressively',
 'aggressive',
 'mr',
 'entry',
 'features',
 'fights',
 'charity',
 'boxing',
 'v',
 'wwe',
 'match',
 'rocky',
 'hulk',
 'hogan',
 'explosive',
 'mix',
 'film',
 'doesnt',
 'slow',
 'minute',
 'not',
 'action',
 'feature',
 'one',
 'saddest',
 'scenes',
 'rocky',
 'films',
 'final',
 'bout',
 'suitably',
 'hard',
 'hitting',
 'loved',
 'final',
 'scene',
 'fans',
 'originals',
 'another',
 'treat',
 'another',
 'entry',
 'holds',
 'well',
 'today']

In [ ]:
vectr = TfidfVectorizer(ngram_range=(1,2),min_df=1)
vect_X = vectr.fit_transform(X_train)